# 💻 Overlay

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/08_overlay.ipynb)

Overlay-analyser er GIS-operasjoner der to eller flere vektorlag er
kombinert for å produsere nye geometrier. Typiske overlay-operasjoner inkluderer *union*,
*intersection*, og *difference* - navngitt etter resultatet av kombinasjonen av to lag. I denne notebooken (inspirert av [Python for Geographic Data Analysis](https://pythongis.org/part2/chapter-06/nb/08-overlay-analysis-with-vector-data.html)) ser vi nærmere på hvordan vi kan utføre overlay operasjoner med vektor data.

Den grunnleggende ideen med vektor-overlay-operasjoner er demonstrert i figuren under, hvor de grønne områdene representerer områdene som utgjør resultatet etter overlay-operasjonen. Det er viktig å huske at overlays opererer på GeoDataFrame-nivå, ikke på individuelle geometrier, og egenskapene fra begge beholdes (ofte, men ikke alltid). I praksis utføres denne operasjonen for hver form i den venstre GeoDataFrame mot hver annen form i den høyre GeoDataFrame.

![Fire paneler som viser union, intersection, symmetric difference og difference av to geometrier.](https://pythongis.org/_images/vector_overlay_processes.png)


Romlig overlay med to inputvektorlag (rektangel, sirkel). Det resulterende vektorlaget vises i grønt. *Kilde: [QGIS dokumentasjon](https://docs.qgis.org/latest/en/docs/gentle_gis_introduction/vector_spatial_analysis_buffers.html#figure-overlay-operations)*
:::

For å demonstrere hvordan disse overlay-operasjonene fungerer i praksis, vil vi utføre vektor-overlay-operasjoner mellom to polygon-datasett som representerer 1) bydelene i Oslo og 2) en 5000-meter buffer rundt Oslo S. La oss starte med å lese inn datasettene og forberede dem for analysen:

In [ ]:
import geopandas
import pathlib 

NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

bydeler = geopandas.read_file(DATA_MAPPE / "oslo_bydeler" /)
oslo_s = geopandas.read_file(DATA_MAPPE / "oslo_bydeler" / )
bydeler.shape

Her kan vi se at bydelene inneholder MultiPolygon-geometrier som representerer til sammen 17 bydeler, mens `oslo_s` representerer et enkelt punkt for Oslo S. Siden vektor-overlay-operasjoner skjer mellom to geografiske datasett, er det nødvendig å sikre at de begge deler samme koordinatsystem og i vårt tilfelle et projisert koordinatsystem, siden vi skal bruke en metrisk buffersone.
La oss derfor først sjekke crs-attributtene og gjøre nødvendige reprojiseringer:

Begge datasettene våre er i `EPSG:4326`, la oss reprojisere til `EPSG:25832` som egner seg for norske data:

In [ ]:
# Reprojiser bydeler


# Reprojiser oslo_s



La oss for sikkerhetsskyld dobbeltsjekke at lagene har samme crs:

In [ ]:
assert bydeler.crs == oslo_s.crs, "Ulike koordinatsystemer!"

Supert, koordinatsystemet (CRS) stemmer overens mellom lagene. Derfor kan vi fortsette og lage en 5 kilometer buffer rundt Oslo S som vi vil bruke i våre vektor-overlay-operasjoner:

Her har vi først lagd en kopi av `oslo_s`-laget vårt, og brukt `.buffer()`-metoden til å lage en buffer Polygon på 5000 meters radius rundt Oslo S. Vi kan nå lage et enkelt plot for å se hvordan lagene overlapper hverandre:

## Intersection

Nå kan vi utføre en overlay-analyse mellom disse to lagene. Vi skal lage et nytt lag som `intersect`("skjærer") med bydelslaget. Vi bruker da metoden `.overlay()` i GeoPandas til å gjøre analysen mellom laget `bydeler` og `oslo_s_buffer`. Parameteren `how` i `overlay()`-metoden angir hvilken type overlay-analyse som utføres. De ulike metodene man kan velge mellom er `intersection`, `union`, `symmetric_difference`, `difference` og `identity`.

Vi begynner med å bruke `intersection` som overlay-metode:

In [ ]:
# Intersection


Vi kan nå se at vi har en GeoDataFrame med 13 rader for alle bydelene som intersecter med bufferlaget vårt. Vi kan også se at vi har med kolonner fra begge datasettene, og dermed at overaly-metoden fungerer litt likt `sjoin()` som vi brukte i en [tidligere forelesning](#05_romlig_kobling).

For å lettere illustrere hvordan de ulike overlay metodene fungerer, lager vi nå en funksjon som plotter et kart før og etter overlay-analysen ved siden av hverandre:

In [ ]:
import matplotlib.pyplot as plt


def plot_overlay(gdf1, gdf2, result, title):
    """
    Creates two maps next to each other based on `gdf1`, `gdf2` and the
    `result` GeoDataFrames.

    Source: https://pythongis.org/part2/chapter-06/nb/08-overlay-analysis-with-vector-data.html#intersection
    """

    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 5))

    ax1 = gdf1.plot(ax=ax1)
    ax1 = gdf2.plot(ax=ax1, color="red", alpha=0.3)

    result.plot(ax=ax2)

    # Fetch bounds and apply to axis 2
    xmin, ymin, xmax, ymax = gdf1.total_bounds

    ax2.set_xlim(xmin, xmax)
    ax2.set_ylim(ymin, ymax)

    fig.suptitle(title, fontsize=16)
    # Add an arrow between the plots
    fig.text(0.49, 0.5, "⇨", fontsize=30, color="red")
    ax1.axis("off")
    ax2.axis("off")
    plt.tight_layout()
    return fig, ax1, ax2

Med denne funksjonen kan vi nå bruke `intersection`-laget vi nettopp lagde og plotte før- og etter-resultatet:

Som vi kan se, beholder "intersection" operasjonen bydelene som krysser sirkelen og beholder alle disse geometriene i resultatet. En viktig ting å merke seg er at med `overlay()` vil den kryssende GeoDataFramen (`oslo_s_buffer`) også modifisere inputgeometriene ved å kutte dem i grenseområdene der de krysser. Dette er en av de viktigste forskjellene mellom `.overlay()` og `sjoin()` metodene, da `sjoin()` ikke vil modifisere inputgeometriene. Som nevnt tidligere, beholdes attributtdata fra begge GeoDataFramene for objektene som er en del av resultatet. I det følgende vil vi vise én etter én hvordan forskjellige overlay-operasjoner (dvs. union, difference, symmetric difference, identity) påvirker resultatene.

## Union

La oss bruke de samme inputdatasettene og utføre en `union`-overlay:

In [ ]:
# Union


Når man bruker `"union"` overlay-operasjonen, beholdes geometriene fra begge GeoDataFrames i resultatet. Som du kan se, har antall rader økt ganske betydelig fra 17 til 22 rader. Dette skjer fordi bydelene igjen blir modifisert av `oslo_s_buffer` i områdene der geometriene krysser hverandre: bydelene blir delt i to i områder der buffergeometrien krysser bydeler-geometriene. Dette vil derfor øke antall rader i det endelige resultatet.

## Difference og symmetric difference

Noen ganger kan det være nyttig å fokusere på å trekke ut geometrier som ligger utenfor et gitt lag. Dette kan oppnås ved å bruke .overlay() med "difference" operatoren:

In [ ]:
# Difference


Symmetric differance-overlay-operasjonen er en interessant en. Den vil beholde geometriene og attributtene utenfor `oslo_s_buffer`-laget, samt opprette en geometri innenfor `oslo_s_buffer` som inkluderer områder som er innenfor `oslo_s_buffer`-ringen, men utenfor `bydeler`-GeoDataFramen. Det vil si, i vårt tilfelle inneholder den et lite stykke av Oslofjorden, som vist nedenfor:

In [ ]:
# Symmetric Difference


Som vi kan se i tabellen, så inneholder tabellen attributter fra bydelene og en rad med data fra `oslo_s_buffer` i den siste raden.

## Overlay med befolkningsdata

I denne notebooken vil vi utføre en overlay-analyse for å velge de polygon-cellene i et grid-datasett som ligger innenfor Oslo. I denne
øvelsen, bruker vi to input-datasett: et grid av statistiske polygoner med
befolkningen i Oslo og et polygon-datasett av norske kommuner
(`kommuner.geojson`), hvor vi skal velge bare Oslo kommune.

In [ ]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas

rutenett = geopandas.read_file(DATA_MAPPE / "ssb_rutenett" / "befolkning_250m_2023_akershus.shp")

kommuner = geopandas.read_file(
    DATA_MAPPE / "kommuner" / "kommuner.geojson"
)

display(kommuner.head())

Vi kan nå velge bare Ås kommune fra kommune-datasettet.

Før vi fortsetter er det en god ide å sørge for at begge lagene har samme koordinatsystem:

In [ ]:
# Sjekke at lagene har samme crs


La oss gjøre en rask overleggsvisualisering av de to lagene:

In [ ]:
# Plott lagene


Her er alle de grå rutene gridceller med befolkningsdata for Akershus fylke. Datasettet inneholder 17376 ruter.
Det blå omrisset representerer Ås kommune. Vårt mål er nå å utføre en overlay-analyse og velge de geometriene fra rutenettet som krysser polygonet for Ås kommune.

Vi vil lage et nytt lag basert på rutenett-polygoner som `intersect`-er med Ås-laget vårt. Vi kan bruke metoden `overlay()` med en `GeoDataFrame` for å utføre overlay-analysen som tar som en input 1) andre GeoDataFrames, og 2)
parameteren `how` som kan brukes til å kontrollere hvordan overlay-analysen blir
utført (mulige verdier er `'intersection'`, `'union'`,`'symmetric_difference'`, `'difference'`, og `'identity'`):

La oss plotte dataene våre og se hva vi har:

Som et resultat har vi nå bare de rutenett cellene som krysser med Helsingfors
grenser. Hvis du ser nøye etter, kan du også observere at **rutenett cellene er
klippet basert på grensen.**

- Hva med dataattributter? La oss se hva vi har:

Som vi kan se, på grunn av overleggsanalysen, inneholder datasettet attributtene
fra begge input lagene.

La oss lagre resultatrutenettet vårt som en GeoPackage.

Det er mange flere eksempler på forskjellige typer overleggsanalyse i
[Geopandas dokumentasjon](http://geopandas.org/set_operations.html) hvor du
kan gå og lære mer.